In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import warnings
warnings.filterwarnings('ignore')

Data Source: https://berean.bible/downloads.htm

In [2]:
import os, sys
os.listdir()

['model.ipynb',
 '.git',
 't_kjv.csv',
 'README.md',
 '.ipynb_checkpoints',
 'model.pkl']

In [3]:
data = pd.read_csv('t_kjv.csv')

In [4]:
data.head()

,id,b,c,v,t
0,1001001,1,1,1,In the beginning God created the heaven and th...
1,1001002,1,1,2,"And the earth was without form, and void; and ..."
2,1001003,1,1,3,"And God said, Let there be light: and there wa..."
3,1001004,1,1,4,"And God saw the light, that it was good: and G..."
4,1001005,1,1,5,"And God called the light Day, and the darkness..."


In [5]:
data.shape

(31103, 5)

In [6]:
read_data = data[['t']]

In [7]:
read_data.head()

,t
0,In the beginning God created the heaven and th...
1,"And the earth was without form, and void; and ..."
2,"And God said, Let there be light: and there wa..."
3,"And God saw the light, that it was good: and G..."
4,"And God called the light Day, and the darkness..."


In [8]:
import nltk

In [9]:
def tokenize(data):
    return nltk.word_tokenize(data)
read_data['tokens'] = read_data['t'].apply(tokenize)

In [10]:
read_data.head()

,t,tokens
0,In the beginning God created the heaven and th...,"[In, the, beginning, God, created, the, heaven..."
1,"And the earth was without form, and void; and ...","[And, the, earth, was, without, form, ,, and, ..."
2,"And God said, Let there be light: and there wa...","[And, God, said, ,, Let, there, be, light, :, ..."
3,"And God saw the light, that it was good: and G...","[And, God, saw, the, light, ,, that, it, was, ..."
4,"And God called the light Day, and the darkness...","[And, God, called, the, light, Day, ,, and, th..."


# using brown pos tagger

In [11]:
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories = 'news')
brown_sents = brown.sents(categories = 'news')

In [12]:
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]
unigram_tagger = nltk.UnigramTagger(train_sents)
unigram_tagger.evaluate(test_sents)

0.8121200039868434

In [13]:
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff = t0)
model = nltk.BigramTagger(train_sents, backoff = t1)

In [14]:
model.evaluate(test_sents)

0.8452108043456593

In [15]:
from pickle import dump
output = open('model.pkl','wb')
dump(model, output, -1)
output.close()

# model predict

In [16]:
def tagger(data):
    tokens = data.split()
    return model.tag(tokens)
read_data['tokens with tag'] = read_data['t'].apply(tagger)

In [17]:
read_data['tokens with tag'][0]

[('In', 'IN'),
 ('the', 'AT'),
 ('beginning', 'NN'),
 ('God', 'NP-TL'),
 ('created', 'VBN'),
 ('the', 'AT'),
 ('heaven', 'NN'),
 ('and', 'CC'),
 ('the', 'AT'),
 ('earth.', 'NN')]

In [18]:
classes = []

def decode_classes(data):
    for i,j in data:
        if j in classes:
            pass
        else:
            classes.append(j)
    return

out = read_data['tokens with tag'].apply(decode_classes)

In [19]:
number_of_target_class = len(classes)
print(number_of_target_class)

105


# encode data

In [20]:
diction_pos = {}

for count,i in enumerate(classes):
    diction_pos[i] = count
    


In [40]:
count = 0
for i,j in diction_pos.items():
    print(i,j)
    
    if count == 5:
        break
    else:
        count += 1
        continue
        

IN 0
AT 1
NN 2
NP-TL 3
VBN 4
CC 5


In [22]:
list_me = [1,2,3,4,5,6,9]
list_me[1:-1]

[2, 3, 4, 5, 6]

In [143]:
x, y = [],[]

search = 7

def search_key(value):
    found = [i for i,j in diction_pos.items() if j == value]
    return found[0]

# 106 means just starting
# 107 means full stop/end statement

def extract_data(data):
    global x
    global y
    try:
        x.append(106)
        y.append(diction_pos[data[1][1]])
        count = 1
        data_len = len(data[1:-1])
        for i,j in data[1:-1]:
            x.append(diction_pos[j])
            y.append(diction_pos[data[count+1][1]])
            count += 1
            if count == data_len:
                x.append(y[-1])
                y.append(107)
                break
    except IndexError as err:
        x = x[:-1]
        pass
    return 

In [144]:
diction_pos['PPSS']

33

In [145]:
read_data['tokens with tag'][:2]

0    [(In, IN), (the, AT), (beginning, NN), (God, N...
1    [(And, CC), (the, AT), (earth, NN), (was, BEDZ...
Name: tokens with tag, dtype: object

In [146]:
out = read_data['tokens with tag'].apply(extract_data)

In [147]:
len(x), len(y)

(758531, 758531)

In [149]:
count = 1
for i, j in zip(x[200:],y[200:]):
    print(i,j)
    if count == 10:
        break
    count += 1

19 2
2 1
1 2
2 21
21 2
2 5
5 1
1 2
2 2
2 21
